# Comparison with SPOTDis

In [ ]:
import sys
import joblib
sys.modules['sklearn.externals.joblib'] = joblib
import hdbscan
import matplotlib.pylab as plt
import numpy as np

from sklearn.manifold import TSNE
from spikeship import spikeship, tools
from spot import spot

### Reading data sources

In [ ]:
spike_times    = np.load('../example_data/spotdis_simulations/spike_times.npy')
ii_spike_times = np.load('../example_data/spotdis_simulations/ii_spike_times.npy')
epoch_labels   = np.load('../example_data/spotdis_simulations/stim_label.npy')
print ("ii_spike_times.shape", ii_spike_times.shape)
print ("# epochs =", ii_spike_times.shape[0])
print ("# neurons =",ii_spike_times.shape[1])

spot_dissimilarities = None; spikeship_diss = None; G = None;

### SPOTDis & SpikeShip computations

In [ ]:
%%time
spot_diss = spot.distances(spike_times, ii_spike_times)

In [ ]:
%%time
spikeship_diss = spikeship.distances(spike_times, ii_spike_times)

In [ ]:
plt.clf()
alpha = 0.5; marker = 'o';
tPlot,axes = plt.subplots(
    figsize=(10.5,4), facecolor='w',nrows=1, ncols=2, sharex=False, sharey=False, constrained_layout=True)

# true labels sorting
si = np.argsort(epoch_labels)
print ("epoch_labels shape:", si.shape)

# dissimilarity matrices (vis)
im0 = axes[0].imshow(spot_diss[:, si][si], cmap='PuBu')

axes[0].set_xlabel('Epoch',fontsize=13); axes[0].set_ylabel('Epoch',fontsize=13);
plt.colorbar(im0, ax=axes[0], label='SPOTDis')

im1 = axes[1].imshow(spikeship_diss[:, si][si], cmap='PuBu')

axes[1].set_xlabel('Epoch',fontsize=13); axes[1].set_ylabel('Epoch',fontsize=13);
_ = plt.colorbar(im1, ax=axes[1], label='SpikeShip')

SPOTDis has a computational complexity of $\mathcal{O}(M^2N^2n^2)$, where $M$ is the number of epochs or trials, $N$ is the number of active Neurons, and $n$ the average spikes per epoch. In contrast, SpikeShip has a computational complexity of $\mathcal{O}(M^2 N n)$ (i.e., quadratic speed-up).

---

### Visualization of clusters by using SPOTDis and SpikeShip dissimilarity measures

In [ ]:
clusterer = hdbscan.HDBSCAN(metric='precomputed')
spot_labels = clusterer.fit_predict(spot_diss)

clusterer = hdbscan.HDBSCAN(metric='precomputed')
spikeship_labels = clusterer.fit_predict(spikeship_diss)

print(len(np.unique(spot_labels)), 'clusters found (SPOTDis)')
print(len(np.unique(spikeship_labels)), 'clusters found (SpikeShip, neuron-specific flows)')

In [ ]:
spot_embedding = TSNE(metric='precomputed', n_components=2).fit_transform(spot_diss)
spikeship_embedding   = TSNE(metric='precomputed', n_components=2).fit_transform(spikeship_diss)

In [ ]:
tPlot,axes = plt.subplots(
    figsize=(10,3.7), facecolor='w',nrows=1, ncols=3, sharex=False, sharey=False, constrained_layout=True)

plt.suptitle('2D t-SNE embeddings',fontsize=20)
axes[0].set_title('True clusters')
axes[0].scatter(spot_embedding[:, 0], spot_embedding[:, 1], cmap='Set1_r', c=epoch_labels)

axes[1].set_title('SPOTDis clusters')
axes[1].scatter(spot_embedding[:, 0], spot_embedding[:, 1], cmap='Set1_r', c=spot_labels)

axes[2].set_title('SpikeShip clusters')
axes[2].scatter(spikeship_embedding[:, 0], spikeship_embedding[:, 1], cmap='Set1_r', c=spikeship_labels);

---